<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#EDA" data-toc-modified-id="EDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#attribute-selection" data-toc-modified-id="attribute-selection-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>attribute selection</a></span></li><li><span><a href="#Outliers" data-toc-modified-id="Outliers-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Outliers</a></span></li></ul></li><li><span><a href="#Clustering-based" data-toc-modified-id="Clustering-based-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Clustering based</a></span><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>modeling</a></span></li><li><span><a href="#prediction" data-toc-modified-id="prediction-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>prediction</a></span></li><li><span><a href="#evaluation" data-toc-modified-id="evaluation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>evaluation</a></span></li></ul></li><li><span><a href="#LDA-based" data-toc-modified-id="LDA-based-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>LDA based</a></span></li></ul></div>

In [1]:
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn import mixture
from sklearn.cluster import KMeans
from nltk.cluster import KMeansClusterer, cosine_distance
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support, classification_report, roc_curve, auc, precision_recall_curve
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
seeds = 1234

In [3]:
train = pd.read_json('../data/structured_train.json')
test = pd.read_json('../data/structured_test.json')

In [4]:
train = train.groupby('label').sample(50, random_state=seeds)
test = test.groupby('label').sample(50, random_state=seeds)

In [6]:
select_cols = ["global_index", "doc_path", "label", "reply", "reference_one", "reference_two",
               "Subject", "From", "Lines", "Organization", "contained_emails", "long_string", "text", "error_message"]
print("\nmay use cols: \n", select_cols)
train = train[select_cols]
test = test[select_cols]


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


# Clustering based
- Steps:
    1. Transform into TF-IDF matrix
    2. Dimension reduction into 200
    3. Clustering in cosine similarity space (since it is word)
    4. Assign labels with majority vote based on training set labels
    5. Prediction
        1. Transform test set into TF-IDF matrix
        2. Dimension reduction into 200
        3. Make prediction based on the clusters and mapping between clusters and labels from training set
    6. Evaluation
        1. Based on classification report

## modeling

In [21]:
train['reply'] + ' ' + train['reference_one']

0        Archive-name: atheism/resources Alt-atheism-ar...
1        Archive-name: atheism/introduction Alt-atheism...
2         The argument goes as follows: Q-oid quotes ap...
3         It sounds to me like it's just SCREAMING OUT ...
4         Can somebody reconcile the apparent contradic...
                               ...                        
11078     \tThose things, \twhich ye have both learned,...
11079    In article wrote: Please, please don't make Ba...
11080    In article wrote: Jim, please, that's a lame e...
11081     to the person who doesn't have to do them. Ho...
11082     I hope I didn't award custody, Rich. I purpos...
Length: 11083, dtype: object

In [12]:
train_text = train['reply']
train_label = train['label']
test_text  = test['reply']
test_label = test['label']

In [13]:
from sklearn.decomposition import TruncatedSVD


def tfidf_vectorizer(train_text, test_text, min_df=3):
    tfidf_vect = TfidfVectorizer(stop_words="english", min_df=min_df, max_df=0.95)
    dtm_train = tfidf_vect.fit_transform(train_text)
    dtm_test = tfidf_vect.transform(test_text)
    
    word_to_idx = tfidf_vect.vocabulary_
    print("num of words:", len(word_to_idx))
    return dtm_train, dtm_test, word_to_idx, tfidf_vect

def dimension_reduction(dtm, out_dim=200, verbose=0):
    print("Dimension reduction with truncate SVD:")
    print("   input columns with ", dtm.shape[1])
    print("   output columns with ", out_dim)

    transform_mapper = TruncatedSVD(n_components=out_dim)
    dtm = transform_mapper.fit_transform(dtm)
    if verbose > 0:
        print("singular_values_: ", transform_mapper.singular_values_)
    return dtm, transform_mapper

In [14]:
def fit_clustering_model(dtm_train, train_label, num_clusters, metric='Cosine', model='KMeans', repeats=20):
    
    '''

    '''
    assert metric in ['Cosine']
    assert model in ['KMeans']

    # model training
    if model == 'KMeans':
        if metric == 'Cosine':
            clusterer = KMeansClusterer(num_clusters, cosine_distance, repeats=repeats, avoid_empty_clusters=True)
            clusters = clusterer.cluster(dtm_train, assign_clusters=True)
            train_cluster_pred = [clusterer.classify(v) for v in dtm_train]

    elif model == 'GMM':
        pass 
        # GMM model not good in such case
        # clusterer = mixture.GaussianMixture(n_components=num_clusters, n_init=repeats, covariance_type='diag')
        # clusterer.fit(dtm_train)
        # train_cluster_pred = clusterer.predict(dtm_train)
    
    # Maping clusters into labels
    df = pd.DataFrame(list(zip(train_label, train_cluster_pred)), columns=['actual_class', 'cluster'])
    confusion = pd.crosstab(index=df.cluster, columns=df.actual_class)
    clusters_to_labels = confusion.idxmax(axis=1)
    
    print("Cluster to label mapping: ")
    for idx, t in enumerate(clusters_to_labels):
        print("Cluster {} <-> label {}".format(idx, t))
    print("\n")

    return clusterer, clusters_to_labels

def pred_clustering_model(dtm_test, clusterer, clusters_to_labels):
    test_cluster_pred = [clusterer.classify(v) for v in dtm_test]
    predict = [clusters_to_labels[i] for i in test_cluster_pred]
    return predict

In [15]:
dtm_train, dtm_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, min_df=3)
dtm_train, transform_mapper = dimension_reduction(dtm_train)
dtm_test = transform_mapper.transform(dtm_test)

print('dtm_train.shape', dtm_train.shape)
print('dtm_test.shape', dtm_test.shape)

num of words: 24592
Dimension reduction with truncate SVD:
   input columns with  24592
   output columns with  200
dtm_train.shape (11083, 200)
dtm_test.shape (7761, 200)


In [16]:
clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=60, repeats=1)

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\nltk\cluster\util.py:131: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


Cluster to label mapping: 
Cluster 0 <-> label comp.sys.mac.hardware
Cluster 1 <-> label sci.med
Cluster 2 <-> label talk.politics.guns
Cluster 3 <-> label alt.atheism
Cluster 4 <-> label alt.atheism
Cluster 5 <-> label comp.windows.x
Cluster 6 <-> label misc.forsale
Cluster 7 <-> label rec.sport.baseball
Cluster 8 <-> label talk.politics.mideast
Cluster 9 <-> label comp.sys.mac.hardware
Cluster 10 <-> label rec.autos
Cluster 11 <-> label rec.sport.baseball
Cluster 12 <-> label comp.os.ms-windows.misc
Cluster 13 <-> label rec.autos
Cluster 14 <-> label talk.politics.guns
Cluster 15 <-> label misc.forsale
Cluster 16 <-> label sci.space
Cluster 17 <-> label talk.politics.guns
Cluster 18 <-> label comp.graphics
Cluster 19 <-> label soc.religion.christian
Cluster 20 <-> label sci.med
Cluster 21 <-> label rec.autos
Cluster 22 <-> label sci.space
Cluster 23 <-> label misc.forsale
Cluster 24 <-> label sci.electronics
Cluster 25 <-> label talk.politics.guns
Cluster 26 <-> label talk.politics.m

## prediction

In [17]:
pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)

## evaluation

In [18]:
from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# encoded_test_label = le.fit_transform(test_label)
# print(metrics.classification_report(y_true = encoded_test_label, y_pred=pred, target_names=le.classes_))
print(metrics.classification_report(y_true = test_label, y_pred=pred))

                          precision    recall  f1-score   support

             alt.atheism       0.20      0.21      0.20       319
           comp.graphics       0.41      0.37      0.39       389
 comp.os.ms-windows.misc       0.40      0.52      0.45       394
comp.sys.ibm.pc.hardware       0.41      0.40      0.41       392
   comp.sys.mac.hardware       0.45      0.36      0.40       385
          comp.windows.x       0.85      0.32      0.46       395
            misc.forsale       0.34      0.52      0.42       390
               rec.autos       0.33      0.44      0.38       395
         rec.motorcycles       0.90      0.46      0.61       398
      rec.sport.baseball       0.26      0.64      0.37       397
        rec.sport.hockey       0.00      0.00      0.00       827
               sci.crypt       0.88      0.40      0.55       396
         sci.electronics       0.31      0.27      0.29       393
                 sci.med       0.17      0.51      0.25       198
         

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

# LDA based 